# Import Required Libraries
Import libraries such as PyPDF2, PyMuPDF, PaddleOCR, spaCy, regex, and json.

In [4]:
%pip install spacy


  Using cached numpy-2.2.5-cp310-cp310-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/12.2 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.2 MB 6.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/12.2 MB 7.6 MB/s eta 0:00:02
   ----------- ---------------------------- 3.4/12.2 MB 6.3 MB/s eta 0:00:02
   ------------ --------------------------- 3.7/12.2 MB 5.6 MB/s eta 0:00:02
   ------------- -------------------------- 4.2/12.2 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 4.5/12.2 MB 3.7 MB/s eta 0:00:03
   ---------------- ----------------------- 5.0/12.2 MB 3.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.5/12.2 MB 3.2 MB/s eta 0:00:03
   ------------------- -------------------- 6.0/12.2 MB 3.2 MB/s eta 0:00:02
   --------------------- ------------------ 6.6/12.2 MB 3.1 MB/s eta 0:00:02
   ----------------------- ---------------- 7.1/12.2 MB 3.1 MB/s eta 0:00:02
   ----------

  You can safely remove it manually.
  You can safely remove it manually.


In [3]:
# Import Required Libraries
import PyPDF2
import fitz  # PyMuPDF
from paddleocr import PaddleOCR
import spacy
import re
import json

ModuleNotFoundError: No module named 'spacy'

# Load Sample PDF Files
Load sample PDF files from a predefined directory for testing.

In [ ]:
# Load Sample PDF Files
import os

# Define the directory containing sample PDF files
sample_pdf_dir = "c:/repos/vs-code/baauf-do/repos-py-example/sample_pdfs"

# List all PDF files in the directory
pdf_files = [os.path.join(sample_pdf_dir, file) for file in os.listdir(sample_pdf_dir) if file.endswith(".pdf")]

print(f"Found {len(pdf_files)} PDF files for testing.")

# Determine PDF Type
Implement logic to determine whether the PDF is text-based or scanned/mixed.

In [ ]:
# Determine PDF Type
def determine_pdf_type(pdf_path):
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            if page.get_text():
                return "text-based"
    return "scanned/mixed"

# Test the function on sample PDFs
pdf_types = {pdf: determine_pdf_type(pdf) for pdf in pdf_files}
print(pdf_types)

# Extract Text from Text-Based PDFs
Use PyPDF2 or PyMuPDF to extract text from text-based PDFs.

In [ ]:
# Extract Text from Text-Based PDFs
def extract_text_from_text_based_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Example usage
text_based_pdfs = [pdf for pdf, pdf_type in pdf_types.items() if pdf_type == "text-based"]
for pdf in text_based_pdfs:
    extracted_text = extract_text_from_text_based_pdf(pdf)
    print(f"Extracted text from {pdf}:\n{extracted_text[:500]}...\n")

# Perform OCR on Scanned PDFs
Use PaddleOCR to extract text from scanned or mixed PDFs.

In [ ]:
# Perform OCR on Scanned PDFs
ocr = PaddleOCR(use_angle_cls=True, lang="en")

def perform_ocr_on_scanned_pdf(pdf_path):
    with fitz.open(pdf_path) as pdf:
        images = []
        for page in pdf:
            pix = page.get_pixmap()
            images.append(pix.pil_tobytes())
    
    extracted_text = ""
    for img in images:
        result = ocr.ocr(img, cls=True)
        for line in result[0]:
            extracted_text += line[1][0] + "\n"
    return extracted_text

# Example usage
scanned_pdfs = [pdf for pdf, pdf_type in pdf_types.items() if pdf_type == "scanned/mixed"]
for pdf in scanned_pdfs:
    ocr_text = perform_ocr_on_scanned_pdf(pdf)
    print(f"OCR text from {pdf}:\n{ocr_text[:500]}...\n")

# Post-Process Extracted Text
Apply text normalization, spelling correction, and other post-processing steps.

In [ ]:
# Post-Process Extracted Text
def post_process_text(text):
    # Example: Normalize whitespace and remove special characters
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s.,]", "", text)
    return text.strip()

# Example usage
sample_text = "Example text with   irregular spacing and special characters!@#"
processed_text = post_process_text(sample_text)
print(f"Processed text: {processed_text}")

# Extract Key Information
Use regex and NLP libraries like spaCy to extract key information such as hotel name, address, dates, and terms.

In [ ]:
# Extract Key Information
nlp = spacy.load("en_core_web_sm")

def extract_key_information(text):
    doc = nlp(text)
    extracted_info = {
        "entities": [(ent.text, ent.label_) for ent in doc.ents],
        "dates": re.findall(r"\d{4}-\d{2}-\d{2}", text),
    }
    return extracted_info

# Example usage
sample_text = "The contract was signed on 2023-10-01 by Hotel ABC located at 123 Main Street."
key_info = extract_key_information(sample_text)
print(f"Extracted key information: {key_info}")

# Generate JSON Output
Format the extracted information into a JSON structure based on the predefined schema.

In [ ]:
# Generate JSON Output
def generate_json_output(extracted_info):
    return json.dumps(extracted_info, indent=4)

# Example usage
json_output = generate_json_output(key_info)
print(f"JSON Output:\n{json_output}")

# Save Results
Save the JSON output to a file and optionally log the results.

In [ ]:
# Save Results
output_dir = "c:/repos/vs-code/baauf-do/repos-py-example/output"
os.makedirs(output_dir, exist_ok=True)

def save_results(json_data, filename):
    output_path = os.path.join(output_dir, filename)
    with open(output_path, "w") as f:
        f.write(json_data)
    print(f"Results saved to {output_path}")

# Example usage
save_results(json_output, "extracted_info.json")